# 3 month visit compliance rate Analysis


## Setup & Imports

In [4]:
# Import all required libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine, text
from pymongo import MongoClient
from bson.objectid import ObjectId
import os
import glob
from dotenv import load_dotenv
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Visualization settings
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

print("✅ All libraries imported")

✅ All libraries imported


## Step 1: Get Clinic Event Details and Filter for No Show patients


In [5]:
import altair as alt
import io
import glob

# ---------------------------------------------------------
# 1. Load Data from All Months
# ---------------------------------------------------------
metadata_dir = 'metadata/'
files = glob.glob(os.path.join(metadata_dir, "DM2_Protocol_Metrics_*.csv"))

data_frames = []

# Map filenames to Month/Date for sorting
# We'll parse the month name from the end of the filename.
month_map = {
    'may': '2024-05-01',
    'june': '2024-06-01',
    'july': '2024-07-01',
    'august': '2024-08-01',
    'september': '2024-09-01',
    'october': '2024-10-01'
}

print(f"Found {len(files)} files in {metadata_dir}")

for file in files:
    try:
        # Extract month name (e.g., 'october' from '...-october.csv')
        month_name = file.split('-')[-1].replace('.csv', '')
        
        if month_name not in month_map:
            print(f"Skipping file with unknown month: {file}")
            continue
            
        df = pd.read_csv(file)
        
        # Select and Rename Columns
        required_cols = ['Zone', 'Medical Org Name', 'Six Month Program Active Patient Count', 'Three Month Visit Compliant Count']
        df = df[required_cols].rename(columns={
            'Six Month Program Active Patient Count': 'Active_Count',
            'Three Month Visit Compliant Count': 'Compliant_Count'
        })
        
        # Add Date/Month column
        df['Date'] = pd.to_datetime(month_map[month_name])
        df['Month'] = month_name.capitalize()
        
        # Handle Duplicates per month
        if not df['Medical Org Name'].is_unique:
             df = df.drop_duplicates(subset=['Medical Org Name'])
             
        data_frames.append(df)
        
    except Exception as e:
        print(f"Error reading {file}: {e}")

if not data_frames:
    print("No data loaded!")
else:
    df_all = pd.concat(data_frames, ignore_index=True)
    
    # Calculate Compliance %
    df_all['Compliance_Pct'] = (df_all['Compliant_Count'] / df_all['Active_Count'].replace(0, 1)) * 100
    
    print(f"Loaded {len(df_all)} records across {df_all['Month'].nunique()} months.")
    display(df_all.head())

    # ---------------------------------------------------------
    # 2. Interactive Visualization
    # ---------------------------------------------------------
    
    # Create a Month Selection Dropdown
    month_options = sorted(list(month_map.keys()), key=lambda x: month_map[x])
    month_options_cap = [m.capitalize() for m in month_options]
    
    month_dropdown = alt.binding_select(options=month_options_cap, name="Select Month: ")
    month_selection = alt.selection_point(fields=['Month'], bind=month_dropdown, value='October')
    
    # Create Clinic Selection (click on scatter plot to filter trends)
    clinic_selection = alt.selection_point(fields=['Medical Org Name'], on='click', empty=True)
    
    color_scale = alt.Scale(domain=['Sac', 'Non-Sac'], range=['#ff7f0e', '#1f77b4'])

    # --- Chart 1: Scatter Plot (Filtered by Month) ---
    scatter = alt.Chart(df_all).mark_circle(size=100).encode(
        x=alt.X('Active_Count:Q', title='Patient Volume'),
        y=alt.Y('Compliance_Pct:Q', title='Compliance %', scale=alt.Scale(domain=[-5, 105])),
        color=alt.Color('Zone:N', scale=color_scale),
        tooltip=['Medical Org Name', 'Active_Count', 'Compliance_Pct', 'Month'],
        opacity=alt.condition(clinic_selection, alt.value(1), alt.value(0.2))
    ).add_params(
        month_selection,
        clinic_selection
    ).transform_filter(
        month_selection
    ).properties(
        title="Clinic Performance (Select Month)",
        width=400,
        height=400
    )
    
    # --- Chart 2: Trend Lines (Filtered by Clinic Selection) ---
    # We show trends for ALL months, but filtered by the clinic selected in Chart 1
    
    base_trend = alt.Chart(df_all).encode(
        x=alt.X('Date:T', title='Month', axis=alt.Axis(format='%b %Y')),
        color=alt.Color('Zone:N', scale=color_scale)
    ).transform_filter(
        clinic_selection
    )

    trend_compliance = base_trend.mark_line(point=True).encode(
        y=alt.Y('Compliance_Pct:Q', title='Compliance %', scale=alt.Scale(domain=[0, 105])),
        tooltip=['Month', 'Compliance_Pct', 'Medical Org Name']
    ).properties(
        title="Compliance Trend (Selected Clinic)",
        width=400,
        height=150
    )
    
    trend_volume = base_trend.mark_line(point=True, strokeDash=[5,5]).encode(
        y=alt.Y('Active_Count:Q', title='Patient Active Count'),
        tooltip=['Month', 'Active_Count', 'Medical Org Name']
    ).properties(
        title="Patient Volume Trend (Selected Clinic)",
        width=400,
        height=150
    )
    
    # Combine Charts
    dashboard = alt.hconcat(
        scatter,
        alt.vconcat(trend_compliance, trend_volume)
    ).resolve_scale(
        color='shared'
    )
    
    display(dashboard)


Found 6 files in metadata/
Loaded 481 records across 6 months.


,Zone,Medical Org Name,Active_Count,Compliant_Count,Date,Month,Compliance_Pct
0,NaN,NaN,2,0,2024-09-01,September,0.0
1,Non-Sac,"62 - B CA/San Jose (Downtown) - Dr. Nguyen, Tam",2,1,2024-09-01,September,50.0
2,Non-Sac,"107 - CA/San Jose - Dr. Tran, John",2,0,2024-09-01,September,0.0
3,Non-Sac,19 - NC/Davidson - Davidson Family Medicine,3,0,2024-09-01,September,0.0
4,Non-Sac,96 - LA/New Orleans - St. Thomas CHC,3,0,2024-09-01,September,0.0


alt.HConcatChart(...)